# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/city_data.csv")

cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Codrington,-38.2667,141.9667,50.68,86,50,15.28,AU,10-19-2021
1,Westerland,54.9079,8.3033,59.18,100,90,21.85,DE,10-19-2021
2,Cape Town,-33.9258,18.4232,66.34,61,0,1.99,ZA,10-19-2021
3,Kulhudhuffushi,6.6221,73.0700,81.88,74,44,11.25,MV,10-19-2021
4,Atuona,-9.8000,-139.0333,78.39,75,26,21.00,PF,10-19-2021
...,...,...,...,...,...,...,...,...,...
559,Ile-Ife,7.4667,4.5667,73.63,96,72,2.98,NG,10-19-2021
560,Constitución,-35.3333,-72.4167,66.02,58,4,5.99,CL,10-19-2021
561,Sambava,-14.2667,50.1667,72.81,83,63,7.78,MG,10-19-2021
562,Trojes,41.7560,19.9620,52.29,69,18,4.79,AL,10-19-2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)

In [24]:
#variables
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)
humidity_heat_fig = gmaps.figure(center=(16.522731, -5.042275), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

humidity_heat_fig.add_layer(heat_layer)

humidity_heat_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
preferred_cities_df = cities_df.loc[(cities_df["Wind Speed"]<= 10) & (cities_df["Cloudiness"] == 0) & 
                                (cities_df["Max Temp"]>= 70) & (cities_df["Max Temp"]<= 80)].dropna()

preferred_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
127,Camacha,33.0833,-16.3333,72.82,73,0,2.30,PT,10-19-2021
175,Colares,38.7992,-9.4469,74.79,65,0,3.00,PT,10-19-2021
266,Mach,29.8637,67.3298,71.62,8,0,9.33,PK,10-19-2021
300,Ardakān,32.3100,54.0175,70.18,13,0,3.74,IR,10-19-2021
307,Mehrān,33.1222,46.1646,75.20,31,0,8.61,IR,10-19-2021
323,Parvatsar,26.8833,74.7667,77.49,36,0,4.21,IN,10-19-2021
346,Avera,33.1940,-82.5271,75.07,32,0,2.64,US,10-19-2021
348,Santa Maria,-29.6842,-53.8069,71.65,47,0,7.07,BR,10-19-2021
373,Coihueco,-36.6167,-71.8333,75.94,44,0,2.30,CL,10-19-2021
441,São Marcos,-28.9711,-51.0681,71.20,39,0,6.24,BR,10-19-2021


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#hotel dataframe
hotel_df = preferred_cities_df[["City","Country", "Lat", "Lng"]].copy()

# Add a "Hotel Name" column to datafram
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
127,Camacha,PT,33.0833,-16.3333,
175,Colares,PT,38.7992,-9.4469,
266,Mach,PK,29.8637,67.3298,
300,Ardakān,IR,32.3100,54.0175,
307,Mehrān,IR,33.1222,46.1646,
323,Parvatsar,IN,26.8833,74.7667,
346,Avera,US,33.1940,-82.5271,
348,Santa Maria,BR,-29.6842,-53.8069,
373,Coihueco,CL,-36.6167,-71.8333,
441,São Marcos,BR,-28.9711,-51.0681,


In [9]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [16]:
# Iterate through dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        print("-----------------------------")

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
print("Data Retrieval Complete")
print("-----------------------------")

Results for Index 127: Camacha.
Closest hotel in Camacha is Hotel Porto Santo & Spa.
-----------------------------
Results for Index 175: Colares.
Closest hotel in Colares is Arribas Sintra Hotel.
-----------------------------
Results for Index 266: Mach.
Closest hotel in Mach is Naqeebullah hotel mach نقیب اللہ ہوٹل مچھ.
-----------------------------
Results for Index 300: Ardakān.
Closest hotel in Ardakān is Ardakan Hotel.
-----------------------------
Results for Index 307: Mehrān.
Closest hotel in Mehrān is بوتیک زنانه بهار.
-----------------------------
Results for Index 323: Parvatsar.
Closest hotel in Parvatsar is Shree Hotel.
-----------------------------
Results for Index 346: Avera.
Missing field/result... skipping.
Results for Index 348: Santa Maria.
Closest hotel in Santa Maria is Hotel Morotin.
-----------------------------
Results for Index 373: Coihueco.
Closest hotel in Coihueco is Frutillares Coihueco.
-----------------------------
Results for Index 441: São Marcos.
Cl

In [17]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
127,Camacha,PT,33.0833,-16.3333,Hotel Porto Santo & Spa
175,Colares,PT,38.7992,-9.4469,Arribas Sintra Hotel
266,Mach,PK,29.8637,67.3298,Naqeebullah hotel mach نقیب اللہ ہوٹل مچھ
300,Ardakān,IR,32.3100,54.0175,Ardakan Hotel
307,Mehrān,IR,33.1222,46.1646,بوتیک زنانه بهار
323,Parvatsar,IN,26.8833,74.7667,Shree Hotel
346,Avera,US,33.1940,-82.5271,
348,Santa Maria,BR,-29.6842,-53.8069,Hotel Morotin
373,Coihueco,CL,-36.6167,-71.8333,Frutillares Coihueco
441,São Marcos,BR,-28.9711,-51.0681,Hotel São Marcos


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

humidity_heat_fig.add_layer(marker_layer)

# Display figure
humidity_heat_fig

Figure(layout=FigureLayout(height='420px'))